# Prepare Dataset


In [2]:
ls

sample_data/


In [3]:
!pip install kaggle

In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

In [0]:
!cat .kaggle/kaggle.json

In [0]:
pwd

In [0]:
ls --all

In [0]:
!mkdir ~/.kaggle

In [0]:
!cp .kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!mkdir dataset_cat_dog

In [0]:
cd dataset_cat_dog

In [0]:
!kaggle competitions download -c dogs-vs-cats

In [0]:
ls

In [0]:
!unzip train.zip 

In [0]:
!unzip test1.zip  

In [0]:
rm test1.zip  train.zip

In [0]:
ls train/ -U | head -4

## Saparateing The Data

The data we get from the Kaggle dataset it’s a mixed data. means all the images of dogs and cats are in the same folder. Now let’s separate them into two separate folders.



In [0]:
import shutil
from os import listdir, path

def seperateData(data_dir):
    for filename in listdir(data_dir):
        if path.isfile(path.join(data_dir, filename)):
            tokens = filename.split('.')
            if tokens[-1] == 'jpg':
                image_path = path.join(data_dir, filename)
                if not os.path.exists(path.join(data_dir, tokens[0])):
                    os.makedirs(path.join(data_dir, tokens[0]))
                shutil.copyfile(image_path, path.join(path.join(data_dir, tokens[0]), filename))
                os.remove(image_path)

In [0]:
seperateData("./train")

In [0]:
!ls ./train/

So by reading the filename, we can get if it’s a dog or cat.

Now we can read all the images and store them in a python list. and feed it to the network one by one. loading the entire dataset into your program, it will occupy too much ram probably in GB, and we don’t even need all the images at the same time. 

So we will create a Class which will get few of those images in batch, let’s say 20 images and then after we train our network on.



Then the generator we collect next set of images from the folder and create another mini batch. this will continue until we are finished with all the images in the folder



## Let’s Create a Dataset Generator


In [0]:
import cv2 # to load the images
import numpy as np # to do matrix mnupulations 
from os.path import isfile, join # to manupulate file paths
from os import listdir # get list of all the files in a directory
from random import shuffle # shuffle the data (file paths)

In [0]:
class DataSetGenerator:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.data_labels = self.get_data_labels()
        self.data_info = self.get_data_paths()

    def get_data_labels(self):
        data_labels = []
        for filename in listdir(self.data_dir):
            if not isfile(join(self.data_dir, filename)):
                data_labels.append(filename)
        return data_labels

    def get_data_paths(self):
        data_paths = []
        for label in self.data_labels:
            img_lists=[]
            path = join(self.data_dir, label)
            for filename in listdir(path):
                tokens = filename.split('.')
                if tokens[-1] == 'jpg':
                    image_path=join(path, filename)
                    img_lists.append(image_path)
            shuffle(img_lists)
            data_paths.append(img_lists)
        return data_paths

      
# Now the data_path list should contain two lists one with all the 
# lists of image paths for dog and one with list of all the image 
# paths for cat      


# So we got all the image file paths and the corresponding labels, 
# Now what? How to get the images?, We will be using the Python’s 
# concept of generator, and the concept of yield to create the 
# mini-batches on the fly and delete them after we are done training 
# our network on it.      

    def get_mini_batches(self, batch_size=10, image_size=(200, 200), allchannel=True):
        images = []
        labels = []
        empty=False
        
        # counter for the current iteration for each class.
        counter=0
        
        # each_batch = int(10 / 2) = 5
        # batch_size for each class
        each_batch_size=int(batch_size/len(self.data_info))
        
        # we will break this loop when none of the classes have 
        # any images left to train
        while True:
            
            # inner for loop to get images from each and every class one by one
            for i in range(len(self.data_labels)):
                label = np.zeros(len(self.data_labels),dtype=int)
                label[i] = 1
                
                
                # So before loading any images, we have to check if that
                # class has any image_path left in the list, so we checked 
                # if the length of that list of that particular class is 
                # less than our counter value. if it is then we considered 
                # it as empty and continuing to the next class. if it’s not 
                # we are setting the empty flag as False and loading the image 
                # using cv2.imread() method.

                if len(self.data_info[i]) < counter+1:
                    empty=True
                    continue
                empty=False
                img = cv2.imread(self.data_info[i][counter])
                img = self.resizeAndPad(img, image_size)
                if not allchannel:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    img = np.reshape(img, (img.shape[0], img.shape[1], 1))
                images.append(img)
                labels.append(label)
            
            # counter for the current iteration for each class.
            counter+=1

            if empty:
                break
            # if the iterator is multiple of batch size return the mini batch
            if (counter)%each_batch_size == 0:
                yield np.array(images,dtype=np.uint8), np.array(labels,dtype=np.uint8)
                del images
                del labels
                images=[]
                labels=[]
                
# So here first we checked if we are shrinking the image or enlarging. 
# Cuz the cv2.INTER_AREA method is better for shrinking the image where as  
# cv2.INTER_CUBIC  is better for enlarging the image.

# Next, we are checking if it’s a horizontal image or a vertical image. 
# and padding the image with zeros so that it becomes a square image.

# Then we applied the cv2.resize  method to scale the image 
# according to the given size.
                
                
    def resizeAndPad(self, img, size):
        h, w = img.shape[:2]

        sh, sw = size
        # interpolation method
        if h > sh or w > sw:  # shrinking image
            interp = cv2.INTER_AREA
        else: # stretching image
            interp = cv2.INTER_CUBIC

        # aspect ratio of image
        aspect = w/h

        # padding
        if aspect > 1: # horizontal image
            new_shape = list(img.shape)
            new_shape[0] = w
            new_shape[1] = w
            new_shape = tuple(new_shape)
            new_img=np.zeros(new_shape, dtype=np.uint8)
            h_offset=int((w-h)/2)
            new_img[h_offset:h_offset+h, :, :] = img.copy()

        elif aspect < 1: # vertical image
            new_shape = list(img.shape)
            new_shape[0] = h
            new_shape[1] = h
            new_shape = tuple(new_shape)
            new_img = np.zeros(new_shape,dtype=np.uint8)
            w_offset = int((h-w) / 2)
            new_img[:, w_offset:w_offset + w, :] = img.copy()
        else:
            new_img = img.copy()
        # scale and pad
        scaled_img = cv2.resize(new_img, size, interpolation=interp)
        return scaled_img
                

# Image classifier

In [0]:
ls

In [0]:
import tensorflow as tf 

class NetworkBuilder: 
    def __init__(self): 
        pass
      
    #=========================Convolution Layer=========================#
    
    # feature_size is the size of the kernel
    # input_layer is to attach a new layer in the model
    # strides  [batch step, height step, width step, channel step]
    def attach_conv_layer(self, input_layer, output_size=32, feature_size=(5, 5), strides=[1, 1, 1, 1], padding='SAME', summary=False):
        with tf.name_scope("Convolution") as scope:
            
            # To create the new conv layer we first need the size of the input 
            # image. or more specifically the number of the channel the input 
            # image has. So, to get the size of the last axis, 
            # use input_layer.get_shape().as_list()[-1] 
            input_size = input_layer.get_shape().as_list()[-1]
          
            # weights for conv layer should be in this shape [kernal height, 
            # kernel weight, input channel size, output channels ].
            weights = tf.Variable(tf.random_normal([feature_size[0], feature_size[1], input_size, output_size]), name='conv_weights')
            
            if summary:
                tf.summary.histogram(weights.name, weights)
            
            biases = tf.Variable(tf.random_normal([output_size]),name='conv_biases')
            
            # TensorFlow’s convolutional conv2d operation expects a 4-dimensional 
            # tensor with dimensions corresponding to batch, width, height and channel.

            conv = tf.nn.conv2d(input_layer, weights, strides=strides, padding=padding)+biases
            return conv
    
    #=========================The Pooling Layer=========================#
                                                                                
    # The ksize is the pooling size which should be in this format 
    # [batch, height, width, channel]. We don’t want to merge any pixels 
    # from batch and channel axis. So for default, we put 1 in those positions
    # and 2 in both height and width position. 
    
    
    def attach_pooling_layer(self, input_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME'):
        with tf.name_scope("Pooling") as scope:
            return tf.nn.max_pool(input_layer, ksize=ksize, strides=strides, padding=padding)

    #=========================Activation Layer=========================#
    
    def attach_relu_layer(self, input_layer):
        with tf.name_scope("Activation") as scope:
            return tf.nn.relu(input_layer)

    def attach_sigmoid_layer(self, input_layer):
        with tf.name_scope("Activation") as scope:
            return tf.nn.sigmoid(input_layer)

    def attach_softmax_layer(self, input_layer):
        with tf.name_scope("Activation") as scope:
            return tf.nn.softmax(input_layer)
          
    #=========================Flatten Layer=========================#
    
    # Till now we were working with images, or feature maps which are also an 
    # image in some form, and all are 3D in shape excluding the batch axis but 
    # the fully connected layer only works with 1D so we have to convert that 
    # 3d input to flat 1D input.
    
    def flatten(self, input_layer):
        with tf.name_scope("Flatten") as scope:
            input_size = input_layer.get_shape().as_list()
            
            # calculated the total number of neurons (representing each 
            # pixel values) in the input layer excluding the batch axis.
            new_size = input_size[-1] * input_size[-2] * input_size[-3]
            
            # size of the 1D vector for our fully connected layer. 
            # So now we reshape the input layer to [batchsize, newsize] 
            # where -1 is for batch size which means it can take any value
            return tf.reshape(input_layer, [-1, new_size])
  
  #=========================Dense Layer=========================#
    
          
    def attach_dense_layer(self, input_layer, size, summary=False):
        with tf.name_scope("Dense") as scope:
            input_size = input_layer.get_shape().as_list()[-1]
            weights = tf.Variable(tf.random_normal([input_size, size]), name='dense_weigh')
            if summary:
                tf.summary.histogram(weights.name, weights)
            biases = tf.Variable(tf.random_normal([size]), name='dense_biases')
            dense = tf.matmul(input_layer, weights) + biases
            return dense
          


**About tf.reshape**

[link text](https://www.tensorflow.org/api_docs/python/tf/reshape)

If one component of shape is the special value -1, the size of that dimension is computed so that the total size remains constant. In particular, a shape of [-1] flattens into 1-D. **At most one component of shape can be -1.**



```
# tensor 't' has shape [3, 2, 3]
# pass '[-1]' to flatten 't'
reshape(t, [-1]) ==> [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6]
---

[1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6]

# -1 can also be used to infer the shape

# -1 is inferred to be 9:
reshape(t, [2, -1]) ==> [[1, 1, 1, 2, 2, 2, 3, 3, 3],
                         [4, 4, 4, 5, 5, 5, 6, 6, 6]]
                         
```


# Tensorboard setup

In [0]:
!mkdir tensorboard_tut

In [0]:
cd tensorboard_tut

In [0]:
!mkdir summary_log

In [0]:
ls 

In [0]:
pwd

### Tensorboard Installation

In [0]:
LOG_DIR = "/content/dataset_cat_dog/tensorboard_tut/summary_log/"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
"""Make sure we're able to connect to the TensorBoard service."""
! curl http://localhost:6006

In [0]:
"""## 2
Download and unzip [ngrok](https://ngrok.com).
"""
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1


In [0]:
! unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1

In [0]:
! ls

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


In [0]:
# Install
! npm install -g localtunnel

# Tunnel port 6006 (TensorBoard assumed running)
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')

# Get url
!cat url.txt

### Link to Tensorboard

In [0]:
import tensorflow as tf
a = tf.constant(2)
b = tf.constant(3)

x = tf.add(3,5)

#using with clause 

with tf.Session() as sess:
  writer = tf.summary.FileWriter("/content/dataset_cat_dog/tensorboard_tut/summary_log/", sess.graph)
  print ("value of x : ",sess.run(x))
  
writer.close()

In [0]:
pwd

In [0]:
ls

In [0]:
ls /content/dataset_cat_dog/tensorboard_tut/summary_log/ 

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# Let’s Create a CNN Model in Tensorflow

In [0]:
import tensorflow as tf
import datetime 
import numpy as np 
import os

 let’s create the place holders

In [0]:
with tf.name_scope("Input") as scope:
    input_img = tf.placeholder(dtype='float', shape=[None, 128, 128, 3], name="input")

with tf.name_scope("Target") as scope:
    target_labels = tf.placeholder(dtype='float', shape=[None, 2], name="Targets")


Let’s Design the Model

In [0]:
nb = NetworkBuilder()

with tf.name_scope("ModelV2") as scope:
    model = input_img
    model = nb.attach_conv_layer(model, 32, summary=True)
    model = nb.attach_relu_layer(model)
    model = nb.attach_conv_layer(model, 32, summary=True)
    model = nb.attach_relu_layer(model)
    model = nb.attach_pooling_layer(model)

    model = nb.attach_conv_layer(model, 64, summary=True)
    model = nb.attach_relu_layer(model)
    model = nb.attach_conv_layer(model, 64, summary=True)
    model = nb.attach_relu_layer(model)
    model = nb.attach_pooling_layer(model)

    model = nb.attach_conv_layer(model, 128, summary=True)
    model = nb.attach_relu_layer(model)
    model = nb.attach_conv_layer(model, 128, summary=True)
    model = nb.attach_relu_layer(model)
    model = nb.attach_pooling_layer(model)

    model = nb.flatten(model)
    model = nb.attach_dense_layer(model, 200, summary=True)
    model = nb.attach_sigmoid_layer(model)
    model = nb.attach_dense_layer(model, 32, summary=True)
    model = nb.attach_sigmoid_layer(model)
    model = nb.attach_dense_layer(model, 2)
    prediction = nb.attach_softmax_layer(model)

**Now we will create the optimization and accuracy blocks
**

In [0]:
with tf.name_scope("Optimization") as scope:
    global_step = tf.Variable(0, name='global_step', trainable=False)
    cost = tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=target_labels)
    cost = tf.reduce_mean(cost)
    tf.summary.scalar("cost", cost)

    optimizer = tf.train.AdamOptimizer().minimize(cost,global_step=global_step)

with tf.name_scope('accuracy') as scope:
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(target_labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# let’s train it


In [0]:
pwd

In [0]:
ls /content/dataset_cat_dog/tensorboard_tut/summary_log/

In [0]:
# we need to run this only once
#seperateData("./train")

dg = DataSetGenerator("/content/dataset_cat_dog/train")

In [0]:
epochs = 10
batchSize = 10

saver = tf.train.Saver()
model_save_path="/content/dataset_cat_dog/saved model v2/"
model_name='model'

with tf.Session() as sess:
    summaryMerged = tf.summary.merge_all()

    filename = "/content/dataset_cat_dog/tensorboard_tut/summary_log/run" + datetime.datetime.now().strftime("%Y-%m-%d--%H-%M-%s")
    tf.global_variables_initializer().run()

    if os.path.exists(model_save_path+'checkpoint'):
        saver = tf.train.import_meta_graph('./saved '+modelName+'/model.ckpt.meta')
        saver.restore(sess, tf.train.latest_checkpoint(model_save_path))
    writer = tf.summary.FileWriter(filename, sess.graph)

    for epoch in range(epochs):
        batches = dg.get_mini_batches(batchSize,(128,128), allchannel=True)
        for imgs ,labels in batches:
            imgs=np.divide(imgs, 255)
            error, sumOut, acu, steps,_ = sess.run([cost, summaryMerged, accuracy,global_step,optimizer],
                                            feed_dict={input_img: imgs, target_labels: labels})
            writer.add_summary(sumOut, steps)
            print("epoch=", epoch, "Total Samples Trained=", steps*batchSize, "err=", error, "accuracy=", acu)
            if steps % 100 == 0:
                print("Saving the model")
                saver.save(sess, model_save_path+model_name, global_step=steps)


In [0]:
batches = dg.get_mini_batches(batchSize,(128,128), allchannel=True)
imgs ,labels = batches

In [0]:
imgs.shape

In [0]:
ls

# testing